딥 러닝을 이용한 자연어 처리 입문

20-01 어텐션을 이용한 텍스트 요약(Text Summarization with Attention mechanism)

https://wikidocs.net/72820

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/new_project/project2'
traindata_path = '/content/drive/MyDrive/new_project/project2/traindata/문서요약 텍스트/'
rawdata_path = '/content/drive/MyDrive/new_project/project2/rawdata/'

Mounted at /content/drive


In [ ]:
# from tensorflow.python.client import device_lib
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# print(device_lib.list_local_devices() )

In [ ]:
import tensorflow as tf
tf.__version__

'2.15.0'

In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request
np.random.seed(seed=0)

In [ ]:
data = pd.read_csv(f"{traindata_path}/kobart_train.tsv", nrows = 100000, sep = '\t')
print('전체 리뷰 개수 :',(len(data)))

전체 리뷰 개수 : 100000


In [ ]:
data.columns = ['Text', 'Summary']

In [ ]:
data.head()

,Text,Summary
0,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


In [ ]:
print('Text 열에서 중복을 배제한 유일한 샘플의 수 :', data['Text'].nunique())
print('Summary 열에서 중복을 배제한 유일한 샘플의 수 :', data['Summary'].nunique())

Text 열에서 중복을 배제한 유일한 샘플의 수 : 99952
Summary 열에서 중복을 배제한 유일한 샘플의 수 : 99992


In [ ]:
# text 열에서 중복인 내용이 있다면 중복 제거
data.drop_duplicates(subset=['Text'], inplace=True)
data.drop_duplicates(subset=['Summary'], inplace=True)
print("전체 샘플수 :", len(data))

전체 샘플수 : 99946


In [ ]:
print(data.isnull().sum())

Text       0
Summary    1
dtype: int64


In [ ]:
# Null 값을 가진 샘플 제거
data.dropna(axis=0, inplace=True)
print('전체 샘플수 :',(len(data)))

전체 샘플수 : 99945


In [ ]:
with open(f'{rawdata_path}stopwords_korean.txt', 'r') as file:
    stopwords_korean = file.readlines()

# Remove whitespace and newlines from the stopwords
stopwords_korean = [word.strip() for word in stopwords_korean]
stop_words = set(stopwords_korean)
len(stop_words)

595

In [ ]:
def preprocess_sentence(sentence, remove_stopwords = True):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열  제거 Ex) my husband (and myself) for => my husband for
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거

    #### 이메일 형식 제거
    sentence = re.sub(r'\w+@\w+\.\w+', '', sentence)

    ## 기자
    # sentence = re.sub(r'\*{3}\s*기자\s*', '', sentence)
    # sentence = re.sub(r'\*{2}\s*기자\s*', '', sentence)

    # 불용어 제거 (Text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stop_words if len(word) > 1)
    # 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens

In [ ]:
temp_text = 'AI Hub 문서요약 텍스트에서 학습된 BertSum 기반의 한국어 요약 모델 ... 생성 요약 레이블과 추출 요약 레이블을 모두 가지고 있습니다.'
temp_summary = 'Great way to start (or finish) the day!!!'
print(preprocess_sentence(temp_text))
print(preprocess_sentence(temp_summary, 0))

ai hub 문서요약 텍스트에서 학습된 bertsum 기반의 한국어 요약 모델 ... 생성 요약 레이블과 추출 요약 레이블을 가지고 있습니다.
great way to start the day!!!


In [ ]:
# Text 열 전처리
clean_text = []
for s in data['Text']:
    clean_text.append(preprocess_sentence(s))
clean_text[:5]

['ha당 조사료 400만원…작물별 차등 지원 이성훈 전라남도가 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 생산조정제를 적극 추진키로 했다. 생산조정제는 벼를 심었던 논에 대신 사료작물이나 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다. 올해 전남의 작물 재배 계획면적은 전국 5만ha의 21%인 1만 698ha로, 세부시행지침을 확정, 시군에 통보했다. 지원사업 대상은 2017년산 변동직불금을 받은 농지에 10a 이외 작물을 재배한 농업인이다. 지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 수급 불안 품목은 제외된다. 농지의 경우도 이미 작물 재배 의무가 부여된 간척지, 정부매입비축농지, 농진청 시범사업, 경관보전 직불금 수령 농지 등은 제외될 예정이다. ha당 지원 단가는 평균 340만원으로 사료작물 400만원, 일반작물은 340만원, 콩·팥 두류작물은 280만원 등이다. 벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다. 논에 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다. 전남도는 도와 시군에 관련 기관과 농가 등이 참여하는‘논 타작물 지원사업 추진협의회’를 구성, 지역 특성에 맞는 작목 선정 사업 심의 등을 본격 추진할 방침이다. 최향철 전라남도 친환경농업과장은 “최근 쌀값이 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다”며“쌀 공급과잉을 구조적으로 해결하도록 타작물 재배 지원사업에 많이 참여해주길 바란다”고 말했다.',
 '8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 마무리하며 색채와 빛의 도시를 완성했다. 시에 따르면 사업비 8억원이 투입된 컬러빌리지 사업은‘낮에는 색채, 밤에는 빛’을 주제로 지난해 착공에 들어갔다. 컬러빌리지는 색채를 이용한 경관개선사업으로 사업완료에 고소천사벽화마을과 

In [ ]:
# Summary 열 전처리
clean_summary = []
for s in data['Summary']:
    clean_summary.append(preprocess_sentence(s, 0))
clean_summary[:5]

<ipython-input-13-6085af548d54>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거


["전라남도가 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 대신 사료작물이나 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 '쌀 생산조정제'를 적극적으로 시행하기로 하고 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에서 신청받는다",
 '여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사벽화마을과 자산마을은 알록달록 색깔 옷을 입었고 사업 시행과 준공 과정에서도 주민들의 참여를 유도해 경관사업의 좋은 사례를 만들었다.',
 '전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단 목표 달성을 위한 새해 각오를 다졌다.',
 '광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 지역특화작목 중심으로 농업인 실용교육을 실시한다.',
 '올해 4월과 6월 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항해 전남의 관광지를 방문할 예정이다.']

In [ ]:
data['Text'] = clean_text
data['Summary'] = clean_summary

In [ ]:
data.replace('', np.nan, inplace=True)
print(data.isnull().sum())

Text       0
Summary    0
dtype: int64


In [ ]:
data.dropna(axis = 0, inplace = True)
print('전체 샘플수 :',(len(data)))

전체 샘플수 : 99945


In [ ]:
# 길이 분포 출력
text_len = [len(s.split()) for s in data['Text']]
summary_len = [len(s.split()) for s in data['Summary']]

print('텍스트의 최소 길이 : {}'.format(np.min(text_len)))
print('텍스트의 최대 길이 : {}'.format(np.max(text_len)))
print('텍스트의 평균 길이 : {}'.format(np.mean(text_len)))
print('요약의 최소 길이 : {}'.format(np.min(summary_len)))
print('요약의 최대 길이 : {}'.format(np.max(summary_len)))
print('요약의 평균 길이 : {}'.format(np.mean(summary_len)))


텍스트의 최소 길이 : 26
텍스트의 최대 길이 : 2955
텍스트의 평균 길이 : 205.01888038421131
요약의 최소 길이 : 3
요약의 최대 길이 : 91
요약의 평균 길이 : 25.91522337285507


In [ ]:
text_max_len = 250
summary_max_len = 30

In [ ]:
def below_threshold_len(max_len, nested_list):
  cnt = 0
  for s in nested_list:
    if(len(s.split()) <= max_len):
        cnt = cnt + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (cnt / len(nested_list))))

In [ ]:
below_threshold_len(text_max_len, data['Text'])

전체 샘플 중 길이가 250 이하인 샘플의 비율: 0.7874130771924559


In [ ]:
below_threshold_len(summary_max_len, data['Summary'])

전체 샘플 중 길이가 30 이하인 샘플의 비율: 0.7591375256391015


In [ ]:
data = data[data['Text'].apply(lambda x: len(x.split()) <= text_max_len)]
data = data[data['Summary'].apply(lambda x: len(x.split()) <= summary_max_len)]
print('전체 샘플수 :',(len(data)))

전체 샘플수 : 60212


In [ ]:
# 요약 데이터에는 시작 토큰과 종료 토큰을 추가한다.
data['decoder_input'] = data['Summary'].apply(lambda x : 'sostoken '+ x)
data['decoder_target'] = data['Summary'].apply(lambda x : x + ' eostoken')
data.head()

<ipython-input-26-2597ec9a2495>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['decoder_input'] = data['Summary'].apply(lambda x : 'sostoken '+ x)


,Text,Summary,decoder_input,decoder_target
1,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 여수시는 원도심 일대에서 추진...",여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...,sostoken 여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 ...,여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 전남드래곤즈는 지난 4일 구봉산 해...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...,sostoken 전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,"11~24일, 매실·감·참다래 지역특화작목 이성훈 광양시는 오는 11일부터 24일까...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 지역특화작목 ...","sostoken 광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다...","광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 지역특화작목 ..."
4,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 타이완의 크루즈관광객 ...",올해 4월과 6월 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항해 ...,sostoken 올해 4월과 6월 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이...,올해 4월과 6월 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항해 ...
5,30인 미만 중소기업 사업주에 13만원 지급 이성훈 광양시는 정부가 2018년 1월...,광양시는 30인 미만 중소기업 사업주에 1인당 13만원씩 지급하는 일자리 안정자금 ...,sostoken 광양시는 30인 미만 중소기업 사업주에 1인당 13만원씩 지급하는 ...,광양시는 30인 미만 중소기업 사업주에 1인당 13만원씩 지급하는 일자리 안정자금 ...


In [ ]:
encoder_input = np.array(data['Text'])
decoder_input = np.array(data['decoder_input'])
decoder_target = np.array(data['decoder_target'])

In [ ]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print(indices)

[11299  9105 32745 ... 42613 43567  2732]


In [ ]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [ ]:
n_of_val = int(len(encoder_input)*0.2)
print('테스트 데이터의 수 :',n_of_val)

테스트 데이터의 수 : 12042


In [ ]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [ ]:
print('훈련 데이터의 개수 :', len(encoder_input_train))
print('훈련 레이블의 개수 :',len(decoder_input_train))
print('테스트 데이터의 개수 :',len(encoder_input_test))
print('테스트 레이블의 개수 :',len(decoder_input_test))

훈련 데이터의 개수 : 48170
훈련 레이블의 개수 : 48170
테스트 데이터의 개수 : 12042
테스트 레이블의 개수 : 12042


In [ ]:
src_tokenizer = Tokenizer()
src_tokenizer.fit_on_texts(encoder_input_train)

In [ ]:
threshold = 7
total_cnt = len(src_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in src_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 1184253
등장 빈도가 6번 이하인 희귀 단어의 수: 1065090
단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 119163
단어 집합에서 희귀 단어의 비율: 89.93770756755525
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 19.508439602656075


In [ ]:
src_vocab = 100000
src_tokenizer = Tokenizer(num_words = src_vocab)
src_tokenizer.fit_on_texts(encoder_input_train)

# 텍스트 시퀀스를 정수 시퀀스로 변환
encoder_input_train = src_tokenizer.texts_to_sequences(encoder_input_train)
encoder_input_test = src_tokenizer.texts_to_sequences(encoder_input_test)

In [ ]:
print(encoder_input_train[:3])

[[37366, 31106, 1082, 550, 246, 1058, 708, 1528, 73411, 17744, 31106, 8400, 19254, 25517, 19884, 18641, 14196, 1082, 10172, 255, 3391, 26732, 23999, 1563, 1349, 84, 3138, 1082, 10449, 284, 24492, 135, 86304, 79351, 31107, 4214, 7113, 495, 2123, 73412, 29469, 25518, 6771, 40033, 46985, 797, 8941, 9356, 1, 16, 1082, 10172, 255, 15, 37367, 3125, 3, 3391, 4175, 148, 659, 12097, 8677, 12401, 14, 64148, 3438, 17496, 6104, 1607, 86305, 35047, 26115, 36160, 114, 1954, 5173, 592, 70, 1394, 54162, 3391, 15292, 94876, 4, 2530, 49093, 20221, 2713, 41552, 41553, 135, 1214, 5202, 55, 18, 19885, 3147, 642, 3321, 9977, 8401, 98, 64149, 16964, 1298, 1292, 9102, 3391, 708, 1528, 73411, 17744, 31106, 246, 4644, 35, 1801, 7540, 708, 31108, 374, 1, 6558, 2, 19], [18038, 5044, 7541, 1359, 1208, 24, 177, 43166, 45036, 43166, 43166, 14730, 551, 2209, 73413, 245, 73414, 24, 177, 4026, 981, 43167, 229, 9, 91, 49094, 18039, 43168, 10, 41, 2061, 5044, 9356, 4082, 160, 94877, 6147, 14897, 11810, 416, 7065, 57144, 

In [ ]:
tar_tokenizer = Tokenizer()
tar_tokenizer.fit_on_texts(decoder_input_train)

In [ ]:
threshold = 3
total_cnt = len(tar_tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tar_tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print('단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 %s'%(total_cnt - rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 276059
등장 빈도가 2번 이하인 희귀 단어의 수: 225080
단어 집합에서 희귀 단어를 제외시킬 경우의 단어 집합의 크기 50979
단어 집합에서 희귀 단어의 비율: 81.53329541873295
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 22.448903372394792


In [ ]:
tar_vocab = 50000
tar_tokenizer = Tokenizer(num_words = tar_vocab)
tar_tokenizer.fit_on_texts(decoder_input_train)
tar_tokenizer.fit_on_texts(decoder_target_train)

In [ ]:
decoder_input_train = tar_tokenizer.texts_to_sequences(decoder_input_train)
decoder_target_train = tar_tokenizer.texts_to_sequences(decoder_target_train)
decoder_input_test = tar_tokenizer.texts_to_sequences(decoder_input_test)
decoder_target_test = tar_tokenizer.texts_to_sequences(decoder_target_test)

In [ ]:
drop_train = ([index for index, sentence in enumerate(decoder_input_train) if len(sentence) == 1])
drop_test = ([index for index, sentence in enumerate(decoder_input_test) if len(sentence) == 1])

In [ ]:
# encoder_input_train = np.delete(encoder_input_train, drop_train)
# decoder_input_train = np.delete(decoder_input_train, drop_train)
# decoder_target_train = np.delete(decoder_target_train, drop_train)

# encoder_input_test = np.delete(encoder_input_test, drop_test)
# decoder_input_test = np.delete(decoder_input_test, drop_test)
# decoder_target_test = np.delete(decoder_target_test, drop_test)

In [ ]:
print(len(encoder_input_train))
print(len(drop_train))


48170
0


In [ ]:
# encoder_input_train = np.delete(encoder_input_train, drop_train, axis=0)
# decoder_input_train = np.delete(decoder_input_train, drop_train, axis=0)
# decoder_target_train = np.delete(decoder_target_train, drop_train, axis=0)

# encoder_input_test = np.delete(encoder_input_test, drop_test, axis=0)
# decoder_input_test = np.delete(decoder_input_test, drop_test, axis=0)
# decoder_target_test = np.delete(decoder_target_test, drop_test, axis=0)

# print('훈련 데이터의 개수 :', len(encoder_input_train))
# print('훈련 레이블의 개수 :',len(decoder_input_train))
# print('테스트 데이터의 개수 :',len(encoder_input_test))
# print('테스트 레이블의 개수 :',len(decoder_input_test))

In [ ]:
encoder_input_train = pad_sequences(encoder_input_train, maxlen = text_max_len, padding='post')
encoder_input_test = pad_sequences(encoder_input_test, maxlen = text_max_len, padding='post')
decoder_input_train = pad_sequences(decoder_input_train, maxlen = summary_max_len, padding='post')
decoder_target_train = pad_sequences(decoder_target_train, maxlen = summary_max_len, padding='post')
decoder_input_test = pad_sequences(decoder_input_test, maxlen = summary_max_len, padding='post')
decoder_target_test = pad_sequences(decoder_target_test, maxlen = summary_max_len, padding='post')

In [ ]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
embedding_dim = 128
hidden_size = 256

# 인코더
encoder_inputs = Input(shape=(text_max_len,))

# 인코더의 임베딩 층
enc_emb = Embedding(src_vocab, embedding_dim)(encoder_inputs)

# 인코더의 LSTM 1
encoder_lstm1 = LSTM(hidden_size, return_sequences=True, return_state=True ,dropout = 0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

# 인코더의 LSTM 2
encoder_lstm2 = LSTM(hidden_size, return_sequences=True, return_state=True, dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

# 인코더의 LSTM 3
encoder_lstm3 = LSTM(hidden_size, return_state=True, return_sequences=True, dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

In [ ]:
# 디코더
decoder_inputs = Input(shape=(None,))

# 디코더의 임베딩 층
dec_emb_layer = Embedding(tar_vocab, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)

# 디코더의 LSTM
decoder_lstm = LSTM(hidden_size, return_sequences = True, return_state = True, dropout = 0.4)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state = [state_h, state_c])

In [ ]:
# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation = 'softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_outputs)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 250)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 250, 128)             1280000   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 lstm (LSTM)                 [(None, 250, 256),           394240    ['embedding[0][0]']           
                              (None, 256),                                                        
                              (None, 256)]                                                    

In [ ]:
# 그런데 이번 챕터에서는 어텐션 메커니즘을 사용할 예정이므로 위에서 설계한 출력층을 사용하지 않고, 어텐션 메커니즘이 결합된 새로운 출력층을 설계해보겠습니다. 어텐션 함수를 직접 작성하지 않고 이미 저자의 깃허브에 작성된 어텐션을 사용할 것이므로 아래의 코드를 통해 attention.py 파일을 다운로드하고, AttentionLayer를 임포트합니다. (바다나우 어텐션입니다.)

In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K


class AttentionLayer(Layer):
    """
    This class implements Bahdanau attention (https://arxiv.org/pdf/1409.0473.pdf).
    There are three sets of weights introduced W_a, U_a, and V_a
     """

    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # Create a trainable weight variable for this layer.

        self.W_a = self.add_weight(name='W_a',
                                   shape=tf.TensorShape((input_shape[0][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.U_a = self.add_weight(name='U_a',
                                   shape=tf.TensorShape((input_shape[1][2], input_shape[0][2])),
                                   initializer='uniform',
                                   trainable=True)
        self.V_a = self.add_weight(name='V_a',
                                   shape=tf.TensorShape((input_shape[0][2], 1)),
                                   initializer='uniform',
                                   trainable=True)

        super(AttentionLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, inputs, verbose=False):
        """
        inputs: [encoder_output_sequence, decoder_output_sequence]
        """
        assert type(inputs) == list
        encoder_out_seq, decoder_out_seq = inputs
        if verbose:
            print('encoder_out_seq>', encoder_out_seq.shape)
            print('decoder_out_seq>', decoder_out_seq.shape)

        def energy_step(inputs, states):
            """ Step function for computing energy for a single decoder state
            inputs: (batchsize * 1 * de_in_dim)
            states: (batchsize * 1 * de_latent_dim)
            """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            """ Some parameters required for shaping tensors"""
            en_seq_len, en_hidden = encoder_out_seq.shape[1], encoder_out_seq.shape[2]
            de_hidden = inputs.shape[-1]

            """ Computing S.Wa where S=[s0, s1, ..., si]"""
            # <= batch size * en_seq_len * latent_dim
            W_a_dot_s = K.dot(encoder_out_seq, self.W_a)

            """ Computing hj.Ua """
            U_a_dot_h = K.expand_dims(K.dot(inputs, self.U_a), 1)  # <= batch_size, 1, latent_dim
            if verbose:
                print('Ua.h>', U_a_dot_h.shape)

            """ tanh(S.Wa + hj.Ua) """
            # <= batch_size*en_seq_len, latent_dim
            Ws_plus_Uh = K.tanh(W_a_dot_s + U_a_dot_h)
            if verbose:
                print('Ws+Uh>', Ws_plus_Uh.shape)

            """ softmax(va.tanh(S.Wa + hj.Ua)) """
            # <= batch_size, en_seq_len
            e_i = K.squeeze(K.dot(Ws_plus_Uh, self.V_a), axis=-1)
            # <= batch_size, en_seq_len
            e_i = K.softmax(e_i)

            if verbose:
                print('ei>', e_i.shape)

            return e_i, [e_i]

        def context_step(inputs, states):
            """ Step function for computing ci using ei """

            assert_msg = "States must be an iterable. Got {} of type {}".format(states, type(states))
            assert isinstance(states, list) or isinstance(states, tuple), assert_msg

            # <= batch_size, hidden_size
            c_i = K.sum(encoder_out_seq * K.expand_dims(inputs, -1), axis=1)
            if verbose:
                print('ci>', c_i.shape)
            return c_i, [c_i]

        fake_state_c = K.sum(encoder_out_seq, axis=1)
        fake_state_e = K.sum(encoder_out_seq, axis=2)  # <= (batch_size, enc_seq_len, latent_dim

        """ Computing energy outputs """
        # e_outputs => (batch_size, de_seq_len, en_seq_len)
        last_out, e_outputs, _ = K.rnn(
            energy_step, decoder_out_seq, [fake_state_e],
        )

        """ Computing context vectors """
        last_out, c_outputs, _ = K.rnn(
            context_step, e_outputs, [fake_state_c],
        )

        return c_outputs, e_outputs

    def compute_output_shape(self, input_shape):
        """ Outputs produced by the layer """
        return [
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[1][2])),
            tf.TensorShape((input_shape[1][0], input_shape[1][1], input_shape[0][1]))
        ]

In [ ]:
# 어텐션 층(어텐션 함수)
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# 어텐션의 결과와 디코더의 hidden state들을 연결
decoder_concat_input = Concatenate(axis = -1, name='concat_layer')([decoder_outputs, attn_out])

# 디코더의 출력층
decoder_softmax_layer = Dense(tar_vocab, activation='softmax')
decoder_softmax_outputs = decoder_softmax_layer(decoder_concat_input)

# 모델 정의
model = Model([encoder_inputs, decoder_inputs], decoder_softmax_outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 250)]                0         []                            
                                                                                                  
 embedding (Embedding)       (None, 250, 128)             1280000   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 lstm (LSTM)                 [(None, 250, 256),           394240    ['embedding[0][0]']           
                              (None, 256),                                                        
                              (None, 256)]                                                  

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 2)

history = model.fit(x = [encoder_input_train, decoder_input_train], y = decoder_target_train, \
          validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
          batch_size = 256, callbacks=[es], epochs = 20)

Epoch 1/20
189/189 [==============================] - 222s 1s/step - loss: 6.0397 - val_loss: 5.2607
Epoch 2/20
189/189 [==============================] - 218s 1s/step - loss: 5.5473 - val_loss: 5.2181
Epoch 3/20
189/189 [==============================] - 220s 1s/step - loss: 5.4940 - val_loss: 5.1646
Epoch 4/20
189/189 [==============================] - 221s 1s/step - loss: 5.4573 - val_loss: 5.1392
Epoch 5/20
189/189 [==============================] - 221s 1s/step - loss: 5.4328 - val_loss: 5.1206
Epoch 6/20
189/189 [==============================] - 221s 1s/step - loss: 5.4126 - val_loss: 5.1144
Epoch 7/20
189/189 [==============================] - 222s 1s/step - loss: 5.3937 - val_loss: 5.0947
Epoch 8/20
189/189 [==============================] - 222s 1s/step - loss: 5.3759 - val_loss: 5.0862
Epoch 9/20
189/189 [==============================] - 221s 1s/step - loss: 5.3582 - val_loss: 5.0694
Epoch 10/20
189/189 [==============================] - 221s 1s/step - loss: 5.3409 - val_lo

In [ ]:
model.save("/model/test/test0430.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model

# 모델 로드
loaded_model = load_model("/model/test/test0430.h5")

In [ ]:
src_index_to_word = src_tokenizer.index_word # 원문 단어 집합에서 정수 -> 단어를 얻음
tar_word_to_index = tar_tokenizer.word_index # 요약 단어 집합에서 단어 -> 정수를 얻음
tar_index_to_word = tar_tokenizer.index_word # 요약 단어 집합에서 정수 -> 단어를 얻음

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=[encoder_outputs, state_h, state_c])


In [ ]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(hidden_size,))
decoder_state_input_c = Input(shape=(hidden_size,))

dec_emb2 = dec_emb_layer(decoder_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

In [ ]:
# 어텐션 함수
decoder_hidden_state_input = Input(shape=(text_max_len, hidden_size))
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# 디코더의 출력층
decoder_outputs2 = decoder_softmax_layer(decoder_inf_concat)

# 최종 디코더 모델
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    e_out, e_h, e_c = encoder_model.predict(input_seq)

     # <SOS>에 해당하는 토큰 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = tar_word_to_index['sostoken']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복

        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = tar_index_to_word[sampled_token_index]

        if(sampled_token!='eostoken'):
            decoded_sentence += ' '+sampled_token

        #  <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_token == 'eostoken'  or len(decoded_sentence.split()) >= (summary_max_len-1)):
            stop_condition = True

        # 길이가 1인 타겟 시퀀스를 업데이트
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # 상태를 업데이트 합니다.
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2text(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + src_index_to_word[i]+' '
    return temp

# 요약문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2summary(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=tar_word_to_index['sostoken']) and i!=tar_word_to_index['eostoken']):
            temp = temp + tar_index_to_word[i] + ' '
    return temp

In [ ]:
for i in range(10, 15):
    print("원문 : ",seq2text(encoder_input_test[i]))
    print("실제 요약문 :",seq2summary(decoder_input_test[i]))
    print("예측 요약문 :",decode_sequence(encoder_input_test[i].reshape(1, text_max_len)))
    print("\n")

원문 :  박근혜 정권 시절 벌어진 양승태 사법부의 수많은 피해자를 만들었다 거래를 통해 누군가의 삶은 송두리째 말았다 기자들과 변호사로 구성된 지난 3년간 취재를 통해 사법 민낯을 기록했다 70여명의 피해자들의 목소리를 담았다 저자 박성철 변호사 기자 박상규 기자는 피해자를 만나 국가와 보통 사람의 인생을 바꾸어 보여준다 모든 사건에 연루된 양승태 대법원장의 인생은 살펴본다 책은 2부로 구성돼 있다 1부 법관 시절 청년 법관 일찌감치 정권에 협조하는 판결을 내리며 과정을 설명한다 이를 조작 사건 피해자들의 목소리와 보여준다 2부 시절 2005 사건에서 ktx 승무원 사법부 공개 문건을 통해 드러난 피해자들의 목소리를 담았다 말미에는 해당 사건의 수록했다 독자들이 사건의 흐름을 쉽게 이해할 있도록 돕는다 중간에 변호사 글을 실어 ‘국가 등에 대한 해석을 보탰다 책의 마지막 부분에서는 법이 훼손된 사법 정의를 정립해 나갈 있을지에 대한 고민과 전망을 살펴본다 사진가 찍은 사법 피해자들의 사진을 곳곳에 배치했다 전은재 기자 kr 
실제 요약문 : 변호사 기자 이루어진 지난 3년간 통해 사법 피해자들의 목소리를 담아 엮은 
1/1 [==============================] - 0s 21ms/step
예측 요약문 :  지난 20일 방송된 열린 열린 열린 열린 열린 열린 열린 열린 열린 열린 1 1 1 1 1 1 1 1 밝혔다


원문 :  노인 10명 6명 이상은 근로를 희망하는 것으로 조사됐다 급여는 평균 150만 200만원 수준을 통계청이 23일 발표한 ‘2019년 5월 경제활동인구조사 고령층 결과’에 따르면 55 고령인구 장래 근로를 희망하고 있었다 근로 희망 사유는 ‘일하는 순으로 많았다 남자가 원하는 것으로 조사됐다 장래 근로 희망자는 평균 일할 의사가 있는 것으로 나타났다 70대 중반을 넘어선 75 고령층의 경우는 근로를 희망했다 희망 급여는 150만 200만원이 22 9 로 가장 많았다 200만 250만 17 5 8 5 로 뒤를 이었다 고